In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("copy_text_db.csv", index_col=0)

In [3]:
df.head(5)

,classId,raiting,text
0,222122,1,Феерический бред со спецэфектами.
1,222122,1,Не в коем случае не смотреть в кинотеатре))) с...
2,222122,5,"Офиегнный фильм, один из лучших за послежнее в..."
3,222122,1,Ужасно. Спали полфильма.... Для галочки сходил...
4,222122,1,Тут тома и джери не хватает .


In [4]:
data = df
df = df[df.raiting != 0]
df = df[pd.isnull(df.text) == False]

In [5]:
df.shape

(13483, 3)

In [6]:
df.raiting.describe()

count    13483.000000
mean         4.030557
std          1.334874
min          1.000000
25%          3.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: raiting, dtype: float64

In [7]:
df.raiting.value_counts()

5    7348
4    2704
1    1371
3    1297
2     763
Name: raiting, dtype: int64

In [8]:
df.raiting.value_counts()/df.raiting.count()

5    0.544983
4    0.200549
1    0.101684
3    0.096195
2    0.056590
Name: raiting, dtype: float64

In [9]:
import re
from nltk.corpus import stopwords
import nltk
from nltk.stem.snowball import SnowballStemmer

TAG_RE = re.compile(r'<[^>]+>')
CHARACTER_RE = re.compile(r'[\r\n]')

def remove_tags(text):
    return TAG_RE.sub('', text)

def remove_symbols(text):
    try:
        return re.sub(r'[^а-яА-Яa-zA-Z]+', ' ', text).lower()  
    except TypeError as e:
        print (text)

def stemming(words):
    stemmer = SnowballStemmer("russian")
    return [stemmer.stem(word) for word in words]

def remove_stopwords(words):
    rus_stopwords = stopwords.words("russian")
    return list(filter(lambda x: x not in rus_stopwords, words)) 

def preprocessing(text):
    return " ".join(stemming(remove_stopwords(remove_tags(remove_symbols(text)).split())))

In [10]:
df.text = df.text.map(lambda text: preprocessing(text))

In [11]:
from sklearn.cross_validation import train_test_split
X = df.text
y = df.raiting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
from sklearn import linear_model
from stop_words import get_stop_words

In [13]:
clf = linear_model.LinearRegression()

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
vectorizer = CountVectorizer(analyzer='word', stop_words=get_stop_words("russian"))
vectorizer.fit(X_train)
vec_Xtrain = vectorizer.transform(X_train)
vec_Xtest = vectorizer.transform(X_test)

In [16]:
clf.fit(vec_Xtrain, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
pred_y = clf.predict(vec_Xtest)

In [18]:
from sklearn.metrics import roc_auc_score

In [19]:
pred_y

array([ 3.87111987,  7.4364241 ,  6.32740802, ...,  2.77070418,
        2.14292233,  1.33462067])

In [20]:
pred_y = np.round(pred_y)

In [21]:
pred_y[pred_y < 1] = 0
pred_y[pred_y >=5] = 5

In [22]:
from sklearn.metrics import accuracy_score, recall_score

print('Accuracy', accuracy_score(y_test, pred_y))
print('Recall', recall_score(y_test, pred_y))

Accuracy 0.332134831461
Recall 0.332134831461


//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [22]:
from sklearn.metrics.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from stop_words import get_stop_words

//anaconda/lib/python3.5/site-packages/sklearn/metrics/metrics.py:4: DeprecationWarning: sklearn.metrics.metrics is deprecated and will be removed in 0.18. Please import from sklearn.metrics
  DeprecationWarning)


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
pipeline = Pipeline([
           ('vect', TfidfVectorizer(analyzer = "word", norm='l2', sublinear_tf=True)),
           ('clf', LogisticRegression())
       ])
parameters = {
           'vect__max_df': (0.25, 0.5),
           'vect__ngram_range': ((1, 1), (1, 2)),
           'vect__use_idf': (True, False),
           'clf__C': (0.1, 1, 10),
       }

grid_search = GridSearchCV(pipeline, parameters, n_jobs=3, verbose=1, scoring='accuracy')

In [24]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   45.3s
[Parallel(n_jobs=3)]: Done  72 out of  72 | elapsed:  2.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'vect__use_idf': (True, False), 'clf__C': (0.1, 1, 10), 'vect__max_df': (0.25, 0.5), 'vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1)

In [25]:
print(grid_search.best_score_)
best_parameters = grid_search.best_estimator_.get_params()
print(best_parameters)
for param_name in sorted(parameters.keys()):
        print('\t{0}: {1}'.format(param_name, best_parameters[param_name]))

0.609432082365
{'clf__multi_class': 'ovr', 'vect__use_idf': False, 'vect__vocabulary': None, 'clf__dual': False, 'clf__tol': 0.0001, 'vect__preprocessor': None, 'vect__min_df': 1, 'vect__dtype': <class 'numpy.int64'>, 'vect__norm': 'l2', 'clf__penalty': 'l2', 'vect': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.25, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=False,
        vocabulary=None), 'clf__C': 10, 'clf__intercept_scaling': 1, 'clf__n_jobs': 1, 'clf__random_state': None, 'vect__input': 'content', 'clf__fit_intercept': True, 'clf__max_iter': 100, 'clf': LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_ite

In [26]:
predictions = grid_search.predict(X_test)
print('Accuracy:', accuracy_score(y_test, predictions))
print('Confusion Matrix:\n', confusion_matrix(y_test, predictions))
print('Classification Report:\n', classification_report(y_test, predictions))

Accuracy: 0.611235955056
Confusion Matrix:
 [[ 234   18   33   50  123]
 [  72   21   36   48   71]
 [  41   13   67  137  191]
 [  17    1   27  244  611]
 [  29    1   19  192 2154]]
Classification Report:
              precision    recall  f1-score   support

          1       0.60      0.51      0.55       458
          2       0.39      0.08      0.14       248
          3       0.37      0.15      0.21       449
          4       0.36      0.27      0.31       900
          5       0.68      0.90      0.78      2395

avg / total       0.56      0.61      0.57      4450



In [23]:
sentences = df.text.map(lambda sent: sent.split(" ")).tolist()

In [24]:
from gensim.models import word2vec

In [25]:
num_features = 300                    
min_word_count = 40                        
num_workers = 6
context = 10
downsampling = 1e-3

In [26]:
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

In [27]:
model.init_sims(replace=True)

In [28]:
model_name = "300features_40minwords_10context"
model.save(model_name)

In [29]:
model.syn0.shape

(1893, 300)

In [30]:
#model.index2word

In [31]:
np.zeros((num_features,),dtype="float32")

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [32]:
def makeFeatureVec(words, model, num_features):

    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    
    index2word_set = set(model.index2word)
    
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])

    if nwords != 0:
        featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0.
    
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    for review in reviews:
        if counter % 1000. == 0.:
            print ("Review {0} of {1}".format(counter, len(reviews)))
       
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1.
    return reviewFeatureVecs

In [33]:
train_review = X_train.map(lambda sent: sent.split(" ")).tolist()

In [34]:
test_review = X_test.map(lambda sent: sent.split(" ")).tolist()

In [35]:
train_data_vec = getAvgFeatureVecs( train_review, model, num_features )
test_data_vec = getAvgFeatureVecs( test_review, model, num_features )

Review 0.0 of 9033
Review 1000.0 of 9033
Review 2000.0 of 9033
Review 3000.0 of 9033
Review 4000.0 of 9033
Review 5000.0 of 9033
Review 6000.0 of 9033
Review 7000.0 of 9033
Review 8000.0 of 9033
Review 9000.0 of 9033
Review 0.0 of 4450
Review 1000.0 of 4450
Review 2000.0 of 4450
Review 3000.0 of 4450
Review 4000.0 of 4450


In [41]:

parameters = {
           'C': (0.1, 1, 10)
       }
word2vec_grid_search = GridSearchCV(clf, parameters, n_jobs=3, verbose=1, scoring='accuracy')

In [37]:
y_test[y_test <= 3] = 0
y_test[y_test > 3] = 1

In [38]:
y_train[y_train <= 3] = 0
y_train[y_train > 3] = 1

In [40]:
from sklearn.linear_model import LogisticRegression 
clf = LogisticRegression(C = 1000)
clf.fit(train_data_vec, y_train)
pred = clf.predict(test_data_vec)
accuracy_score(y_test, pred)

0.80966292134831463

In [41]:
from sklearn.ensemble import GradientBoostingClassifier

In [47]:
glf = GradientBoostingClassifier(n_estimators=150)

In [48]:
glf.fit(train_data_vec, y_train)
pred = glf.predict(test_data_vec)
accuracy_score(y_test, pred)


0.79078651685393253

In [58]:
predicted_proba = glf.predict_proba(test_data_vec)[:,1]

In [59]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import roc_auc_score

print('Accuracy', accuracy_score(y_test, pred))
print('Precision', precision_score(y_test, pred))
print('Recall', recall_score(y_test, pred))
print('roc auc', roc_auc_score(y_test, predicted_proba))

Accuracy 0.790786516854
Precision 0.8152
Recall 0.927769347496
roc auc 0.811165940787


In [49]:
train_data_vec

array([[-0.02310327,  0.03365033,  0.04015488, ...,  0.02723235,
        -0.01587757,  0.00157884],
       [-0.01881685, -0.01002743,  0.04712018, ..., -0.00573031,
         0.01614613, -0.00591242],
       [-0.03309727,  0.00631459,  0.02376527, ...,  0.00693041,
        -0.01840356, -0.01322038],
       ..., 
       [-0.0086744 , -0.01208773,  0.0380473 , ...,  0.00178259,
         0.00960937,  0.00191932],
       [ 0.01197862,  0.04057059,  0.06953141, ...,  0.0213905 ,
        -0.00202777, -0.00896287],
       [-0.03549868, -0.00065839,  0.02034354, ...,  0.01196092,
        -0.01034542,  0.00047741]], dtype=float32)

In [44]:
def print_result(y_test, predictions):
    print('Accuracy:', accuracy_score(y_test, predictions))
    print('Confusion Matrix:\n', confusion_matrix(y_test, predictions))
    print('Classification Report:\n', classification_report(y_test, predictions))